# Deploy a large-sized LLM

<div align="left">
<a target="_blank" href="https://console.anyscale.com/template-preview/deployment-serve-llm?file=%252Ffiles%252Flarge-size-llm"><img src="https://img.shields.io/badge/🚀 Run_on-Anyscale-9hf"></a>&nbsp;
<a href="https://github.com/ray-proj    ect/ray/tree/master/doc/source/serve/tutorials/deployment-serve-llm/large-size-llm" role="button"><img src="https://img.shields.io/static/v1?label=&amp;message=View%20On%20GitHub&amp;color=586069&amp;logo=github&amp;labelColor=2f363d"></a>&nbsp;
</div>

A large LLM typically runs on multiple nodes with multiple GPUs, prioritizing peak quality and capability: stronger reasoning, broader knowledge, longer context windows, more robust generalization. When higher latency, complexity, and cost are acceptable trade-offs because you require state-of-the-art results.

This tutorial deploys DeepSeek-R1, a large LLM with 685&nbsp;B parameters, using Ray Serve LLM. For smaller models, see [Deploying a small-sized LLM](https://docs.ray.io/en/latest/serve/tutorials/deployment-serve-llm/small-size-llm/README.html) or [Deploying a medium-sized LLM](https://docs.ray.io/en/latest/serve/tutorials/deployment-serve-llm/medium-size-llm/README.html).

---

## Challenges of large-scale deployments

Deploying a 685&nbsp;B-parameter model like DeepSeek-R1 presents significant technical challenges. At this scale, the model can't fit on a single GPU or even a single node. You must distribute it across multiple GPUs and nodes using *tensor parallelism* (splitting tensors within each layer) and *pipeline parallelism* (spreading layers across devices).  

Deploying a model of this scale normally requires you to manually launch and coordinate multiple nodes, unless you use a managed platform like [Anyscale](https://www.anyscale.com/), which automates cluster scaling and node orchestration. See [Deploy to production with Anyscale Services](#deploy-to-production-with-anyscale-services) for more details.

---

## Configure Ray Serve LLM

A large-sized LLM is typically deployed across multiple nodes with multiple GPUs. To fully utilize the hardware, set `pipeline_parallel_size` to the number of nodes and `tensor_parallel_size` to the number of GPUs per node, which distributes the model’s weights evenly.

Ray Serve LLM provides multiple [Python APIs](https://docs.ray.io/en/latest/serve/api/index.html#llm-api) for defining your application. Use [`build_openai_app`](https://docs.ray.io/en/latest/serve/api/doc/ray.serve.llm.build_openai_app.html#ray.serve.llm.build_openai_app) to build a full application from your [`LLMConfig`](https://docs.ray.io/en/latest/serve/api/doc/ray.serve.llm.LLMConfig.html#ray.serve.llm.LLMConfig) object.

**Optional:** Because Deepseek-R1 is a reasoning model, this tutorial uses vLLM’s built-in reasoning parser to correctly separate its reasoning content from the final response. See [Deploying a reasoning LLM: Parse reasoning outputs](https://docs.ray.io/en/latest/serve/tutorials/deployment-serve-llm/reasoning-llm/README.html#parse-reasoning-outputs).

In [1]:
# serve_deepseek_r1.py
from ray.serve.llm import LLMConfig, build_openai_app

llm_config = LLMConfig(
    model_loading_config=dict(
        model_id="my-deepseek-r1",
        model_source="deepseek-ai/DeepSeek-R1",
    ),
    experimental_configs=dict(num_ingress_replicas=1),
    deployment_config=dict(
        autoscaling_config=dict(
            min_replicas=1,
            max_replicas=1,
        )
    ),
    ### Uncomment if your model is gated and needs your Hugging Face token to access it.
    # runtime_env=dict(env_vars={"HF_TOKEN": os.environ.get("HF_TOKEN")}),
    engine_kwargs=dict(
        max_model_len=16384,
        # Split weights among 8 GPUs in the node
        tensor_parallel_size=8,
        pipeline_parallel_size=2,
        reasoning_parser="deepseek_r1",  # Optional: separate reasoning content from the final answer
    ),
)

app = build_openai_app({"llm_configs": [llm_config]})


INFO 02-06 16:34:18 [__init__.py:220] No platform detected, vLLM is running on UnspecifiedPlatform


2026-02-06 16:34:19,870	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 10.128.5.219:6379...
2026-02-06 16:34:19,882	INFO worker.py:2004 -- Connected to Ray cluster. View the dashboard at https://session-9fyy71sw3bgwajvnjflq7jxd9h.i.anyscaleuserdata.com 
2026-02-06 16:34:19,883	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_b0383f9ef0be47c254dfd96fc70a67b56daa506f.zip' (0.05MiB) to Ray cluster...
2026-02-06 16:34:19,884	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_b0383f9ef0be47c254dfd96fc70a67b56daa506f.zip'.
/home/ray/anaconda3/lib/python3.11/site-packages/ray/_private/worker.py:2052: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
INFO 2026-02-06 16:34:19,893 serve 750734 -- ============== Deployment O

**Note:** Before moving to a production setup, migrate to a [Serve config file](https://docs.ray.io/en/latest/serve/production-guide/config.html) to make your deployment version-controlled, reproducible, and easier to maintain for CI/CD pipelines. See [Serving LLMs - Quickstart Examples: Production Guide](https://docs.ray.io/en/latest/serve/llm/quick-start.html#production-deployment) for an example.

---

## Deploy locally

**Prerequisites**

* Access to GPU compute.
* (Optional) A **Hugging Face token** if using gated models. Store it in `export HF_TOKEN=<YOUR-HUGGINGFACE-TOKEN>`.

**Note:** Depending on the organization, you can usually request access on the model's Hugging Face page. For example, Meta’s Llama models approval can take anywhere from a few hours to several weeks.

**Dependencies:**  
```bash
pip install "ray[serve,llm]"
```

**Beware**: this is an expensive deployment.

---

### Launch

Follow the instructions at [Configure Ray Serve LLM](#configure-ray-serve-llm) to define your app in a Python module `serve_deepseek_r1.py`.  

In a terminal, run:  

In [2]:
!serve run serve_deepseek_r1:app --non-blocking

2026-02-06 16:34:28,137	INFO scripts.py:507 -- Running import path: 'serve_deepseek_r1:app'.
INFO 02-06 16:34:30 [__init__.py:220] No platform detected, vLLM is running on UnspecifiedPlatform
2026-02-06 16:34:31,614	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 10.128.5.219:6379...
2026-02-06 16:34:31,624	INFO worker.py:2004 -- Connected to Ray cluster. View the dashboard at https://session-9fyy71sw3bgwajvnjflq7jxd9h.i.anyscaleuserdata.com 
2026-02-06 16:34:31,625	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_5ede6c2b362631ca823f8f408ee688ece7ecec40.zip' (0.05MiB) to Ray cluster...
2026-02-06 16:34:31,626	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_5ede6c2b362631ca823f8f408ee688ece7ecec40.zip'.
/home/ray/anaconda3/lib/python3.11/site-packages/ray/_private/worker.py:2052: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (defau

Deployment typically takes a few minutes as the cluster is provisioned, the vLLM server starts, and the model is downloaded. 

---

### Send requests

Your endpoint is available locally at `http://localhost:8000` and you can use a placeholder authentication token for the OpenAI client, for example `"FAKE_KEY"`.

Example curl:

In [3]:
!curl -X POST http://localhost:8000/v1/chat/completions \
  -H "Authorization: Bearer FAKE_KEY" \
  -H "Content-Type: application/json" \
  -d '{ "model": "my-deepseek-r1", "messages": [{"role": "user", "content": "What is 2 + 2?"}] }'

{"id":"chatcmpl-c717195a-d909-4693-b492-1f5c49234027","object":"chat.completion","created":1770425795,"model":"my-deepseek-r1","choices":[{"index":0,"message":{"role":"assistant","content":"\n\nThe result of 2 + 2 is **4**. \n\n**Step-by-Step Explanation:**\n1. **Start with the number 2.**\n2. **Add 2 to it:**  \n   2 (first number) + 2 (second number).\n3. **Combine the quantities:**  \n   When you have 2 items and add 2 more, the total becomes 4.\n4. **Verification:**  \n   Using fingers, number lines, or basic arithmetic properties confirms the result. In base 10 (standard numbering), 2 + 2 always equals 4.\n\n**Answer:** 4.","refusal":null,"annotations":null,"audio":null,"function_call":null,"tool_calls":[],"reasoning_content":"Okay, so I need to figure out what 2 plus 2 is. Hmm, let's start by recalling basic addition. I know that when you add two numbers, you're combining their values. So if I have 2 apples and someone gives me 2 more apples, how many apples do I have in total?\n

Example Python:

In [4]:
#client.py
from urllib.parse import urljoin
from openai import OpenAI

API_KEY = "FAKE_KEY"
BASE_URL = "http://localhost:8000"

client = OpenAI(base_url=urljoin(BASE_URL, "v1"), api_key=API_KEY)

response = client.chat.completions.create(
    model="my-deepseek-r1",
    messages=[{"role": "user", "content": "Tell me a joke"}],
    stream=True,
)

# Stream and print JSON
for chunk in response:
    # Stream reasoning content first
    if hasattr(chunk.choices[0].delta, "reasoning_content"):
        data_reasoning = chunk.choices[0].delta.reasoning_content
        if data_reasoning:
            print(data_reasoning, end="", flush=True)
    # Later, stream the final answer
    if hasattr(chunk.choices[0].delta, "content"):
        data_content = chunk.choices[0].delta.content
        if data_content:
            print(data_content, end="", flush=True)

{"asctime": "2026-02-06 16:58:36,204", "levelname": "INFO", "message": "HTTP Request: POST http://localhost:8000/v1/chat/completions \"HTTP/1.1 200 OK\"", "filename": "_client.py", "lineno": 1025, "process": 750734, "job_id": "23000000", "worker_id": "23000000ffffffffffffffffffffffffffffffffffffffffffffffff", "node_id": "1a6ddbbb716b74256e415b58e3dca445abdb4074bbfecbc482406ab0", "timestamp_ns": 1770425916204943427}


Okay, the user wants a joke. Let me think. I need something light and not offensive. Maybe a pun or a play on words. What's a classic setup? Maybe involving animals or everyday situations. How about something with a skeleton? Skeletons are funny because they're spooky but not too scary. Oh, right, the skeleton in the closet joke. Wait, but that's common. Let me see. Why don't skeletons fight each other? Because they don't have the guts! That's a good one. Guts can mean both courage and organs, which they lack. Simple and clean. Alternatively, maybe a math joke? Why was the equal sign so humble? Because he knew he wasn't less than or greater than anyone else. That's also good. But the skeleton one is more universally funny. Let's go with that.


Sure! Here's a lighthearted one:  

Why don’t skeletons fight each other?  
…They don’t have the *guts*!  

(Don’t worry, I’ll stick to my day job. 😅)


---

### Shutdown

Shutdown your LLM service: 

In [5]:
!serve shutdown -y

2026-02-06 16:58:54,931	SUCC scripts.py:774 -- Sent shutdown request; applications will be deleted asynchronously.



---

## Deploy to production with Anyscale services

For production deployment, use Anyscale services to deploy the Ray Serve app to a dedicated cluster without modifying the code. Anyscale provides scalability, fault tolerance, and load balancing, keeping the service resilient against node failures, high traffic, and rolling updates, while also automating multi-node setup and autoscaling for large models like DeepSeek-R1.

**Beware**: this is an expensive deployment. At the time of writing, the deployment cost is around \$110 USD per hour in the `us-west-2` AWS region using on-demand instances. Because this node has a high amount of inter-node traffic, and cross-zone traffic is expensive (around \$0.02 per GB), it's recommended to *disable cross-zone autoscaling*. This demo is pre-configured with cross-zone autoscaling disabled for your convenience.

### Prerequisites

The following template runs only on H100 GPUs in your self-hosted Anyscale cloud, as H100s aren't available in Anyscale’s public cloud. This example uses two nodes of type *8xH100-80&nbsp;GB:208CPU-1830&nbsp;GB* on an AWS cloud.

To provision nodes with 1000 GB of disk capacity, see [Changing the default disk size for GCP clusters](https://docs.anyscale.com/configuration/compute/gcp#disk-size) for Google Cloud Platform (GCP) or [Changing the default disk size for AWS clusters](https://docs.anyscale.com/configuration/compute/aws#disk-size) for Amazon Web Services (AWS). 

---

### Launch the service

Anyscale provides out-of-the-box images (`anyscale/ray-llm`), which come pre-loaded with Ray Serve LLM, vLLM, and all required GPU/runtime dependencies. This makes it easy to get started without building a custom image.

Create your Anyscale service configuration in a new `service.yaml` file:
```yaml
#service.yaml
name: deploy-deepseek-r1
image_uri: anyscale/ray-llm:2.49.0-py311-cu128 # Anyscale Ray Serve LLM image. Use `containerfile: ./Dockerfile` to use a custom Dockerfile.
compute_config:
  auto_select_worker_config: true 
  # Change default disk size to 1000GB
  advanced_instance_config:
    ## AWS ##
    BlockDeviceMappings:
      - Ebs:
        - VolumeSize: 1000
          VolumeType: gp3
          DeleteOnTermination: true
        DeviceName: "/dev/sda1"
    #########
    ## GCP ##
    #instanceProperties:
    #  disks:
    #    - boot: true
    #      auto_delete: true
    #      initialize_params:
    #        - disk_size_gb: 1000
    #########
  
working_dir: .
cloud:
applications:
# Point to your app in your Python module
- import_path: serve_deepseek_r1:app
```

Deploy your service

In [7]:
!anyscale service deploy -f service.yaml

/home/ray/anaconda3/lib/python3.11/site-packages/google/rpc/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
(anyscale +1.4s) Updating existing service 'deploy-deepseek-r1'.
(anyscale +3.8s) Created compute config: 'compute-v1-9806d5b4d0b5a96919faa02648ca9808:1'
(anyscale +3.8s) View the compute config in the UI: 'https://console.anyscale.com/v2/cld_a6j8iubw9rqbyigfwk9fut4amk/compute-configs/cpt_cfy6baqr735ajukmcnd7gchb9d'
(anyscale +4.7s) Uploading local dir '.' to cloud storage.
(anyscale +6.2s) Including workspace-managed pip dependencies.
(anyscale +7.2s) Service 'deploy-deepseek-r1' deployed (version ID: wuiu7uxy).
(anyscale +7.2s) View the service in the UI: 'https://console.anyscale.com/services/service2_18jcv96zi1x56m7h43ccv9av62'
(anyscale +7.2s) Qu

**Note:** If your model is gated, make sure to pass your Hugging Face token to the service with `--env HF_TOKEN=<YOUR_HUGGINGFACE_TOKEN>`

**Custom Dockerfile**  
You can customize the container by building your own Dockerfile. In your Anyscale Service config, reference the Dockerfile with `containerfile` (instead of `image_uri`):

```yaml
# service.yaml
# Replace:
# image_uri: anyscale/ray-llm:2.49.0-py311-cu128

# with:
containerfile: ./Dockerfile
```

See the [Anyscale base images](https://docs.anyscale.com/reference/base-images) for details on what each image includes.

---

### Send requests 

The `anyscale service deploy` command output shows both the endpoint and authentication token:
```console
(anyscale +3.9s) curl -H "Authorization: Bearer <YOUR-TOKEN>" <YOUR-ENDPOINT>
```
You can also retrieve both from the service page in the Anyscale console. Click the **Query** button at the top. See [Send requests](#send-requests) for example requests, but make sure to use the correct endpoint and authentication token.  

---

### Access the Serve LLM dashboard

See [Enable LLM monitoring](#enable-llm-monitoring) for instructions on enabling LLM-specific logging. To open the Ray Serve LLM dashboard from an Anyscale service:
1. In the Anyscale console, go to your **Service** or **Workspace**
2. Navigate to the **Metrics** tab
3. Click **View in Grafana** and click **Serve LLM Dashboard**

---

### Shutdown 
 
Shutdown your Anyscale service:

In [8]:
!anyscale service terminate -n deploy-deepseek-r1

/home/ray/anaconda3/lib/python3.11/site-packages/google/rpc/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
(anyscale +2.4s) Service service2_18jcv96zi1x56m7h43ccv9av62 terminate initiated.
(anyscale +2.4s) View the service in the UI at https://console.anyscale.com/services/service2_18jcv96zi1x56m7h43ccv9av62



---

## Enable LLM monitoring

The *Serve LLM dashboard* offers deep visibility into model performance, latency, and system behavior, including:

* Token throughput (tokens/sec)
* Latency metrics: Time To First Token (TTFT), Time Per Output Token (TPOT)
* KV cache utilization

To enable these metrics, go to your LLM config and set `log_engine_metrics: true`. Ensure vLLM V1 is active with `VLLM_USE_V1: "1"`. 
**Note:** `VLLM_USE_V1: "1"` is the default value with `ray >= 2.48.0` and can be omitted.
```yaml
applications:
- ...
  args:
    llm_configs:
      - ...
        runtime_env:
          env_vars:
            VLLM_USE_V1: "1"
        ...
        log_engine_metrics: true
```

---

## Improve concurrency

Ray Serve LLM uses [vLLM](https://docs.vllm.ai/en/stable/) as its backend engine, which logs the *maximum concurrency* it can support based on your configuration.  

Example log:
```console
INFO 07-30 11:56:04 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 29.06x
```

The following are a few ways to improve concurrency depending on your model and hardware:  

**Reduce `max_model_len`**  
Lowering `max_model_len` reduces the memory needed for KV cache.

**Example**: Running DeepSeek-R1 on 2 nodes with 8xH100-80&nbsp;GB GPUs each:
* `max_model_len = 32,768` → concurrency ≈ 29
* `max_model_len = 16,384` → concurrency ≈ 58

**Use distilled or quantized models**  
Quantizing or distilling your model reduces its memory footprint, freeing up space for more KV cache and enabling more concurrent requests. For example, see [`deepseek-ai/DeepSeek-R1-Distill-Llama-70B`](https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Llama-70B) for a distilled version of DeepSeek-R1.


**Upgrade to GPUs with more memory**  
Some GPUs provide significantly more room for KV cache and allow for higher concurrency out of the box.

**Scale with more replicas**  
In addition to tuning per-replica concurrency, you can scale *horizontally* by increasing the number of replicas in your config.  
Raising the replica count increases the total number of concurrent requests your service can handle, especially under sustained or bursty traffic.
```yaml
deployment_config:
  autoscaling_config:
    min_replicas: 1
    max_replicas: 4
```

*For more details on tuning strategies, hardware guidance, and serving configurations, see [Choose a GPU for LLM serving](https://docs.anyscale.com/llm/serving/gpu-guidance) and [Tune parameters for LLMs on Anyscale services](https://docs.anyscale.com/llm/serving/parameter-tuning).*

---

## Troubleshooting

**Hugging Face auth errors**  
Some models, such as Llama-3.1, are gated and require prior authorization from the organization. See your model’s documentation for instructions on obtaining access.

**Out-Of-Memory errors**  
Out‑of‑memory (OOM) errors are one of the most common failure modes when deploying LLMs, especially as model sizes, and context length increase.  
See [Troubleshooting Guide](https://docs.anyscale.com/overview) for common errors and how to fix them.

---

## Summary

In this tutorial, you deployed a large-sized LLM with Ray Serve LLM, from development to production. You learned how to configure Ray Serve LLM, deploy your service on your Ray cluster, and how to send requests. You also learned how to monitor your app and troubleshoot common issues.